**Examples**<br>
Start by creating a series with 9 one minute timestamps.

In [1]:
import numpy as np
import pandas as pd

In [2]:
index = pd.date_range('1/1/2019', periods=8, freq='T')
series = pd.Series(range(8), index=index)
series

2019-01-01 00:00:00    0
2019-01-01 00:01:00    1
2019-01-01 00:02:00    2
2019-01-01 00:03:00    3
2019-01-01 00:04:00    4
2019-01-01 00:05:00    5
2019-01-01 00:06:00    6
2019-01-01 00:07:00    7
Freq: T, dtype: int64

<img src="https://www.w3resource.com/w3r_images/pandas-series-resample-image-1.svg" width="400" height="200">

Downsample the series into 3 minute bins and sum the values of the timestamps falling into a bin.

In [3]:
series.resample('3T').sum()

2019-01-01 00:00:00     3
2019-01-01 00:03:00    12
2019-01-01 00:06:00    13
Freq: 3T, dtype: int64

Downsample the series into 3 minute bins as above, but label each bin using the right edge instead of the left.<br>

In [4]:
series.resample('3T', label='right').sum()

2019-01-01 00:03:00     3
2019-01-01 00:06:00    12
2019-01-01 00:09:00    13
Freq: 3T, dtype: int64

Downsample the series into 3 minute bins as above, but close the right side of the bin interval.

In [5]:
series.resample('3T', label='right', closed='right').sum()

2019-01-01 00:00:00     0
2019-01-01 00:03:00     6
2019-01-01 00:06:00    15
2019-01-01 00:09:00     7
Freq: 3T, dtype: int64

Upsample the series into 30 second bins.

In [6]:
series.resample('30S').asfreq()[0:5]   # Select first 5 rows

2019-01-01 00:00:00    0.0
2019-01-01 00:00:30    NaN
2019-01-01 00:01:00    1.0
2019-01-01 00:01:30    NaN
2019-01-01 00:02:00    2.0
Freq: 30S, dtype: float64

Upsample the series into 30 second bins and fill the NaN values using the pad method.

In [7]:
series.resample('30S').pad()[0:5]

2019-01-01 00:00:00    0
2019-01-01 00:00:30    0
2019-01-01 00:01:00    1
2019-01-01 00:01:30    1
2019-01-01 00:02:00    2
Freq: 30S, dtype: int64

Upsample the series into 30 second bins and fill the NaN values using the bfill method.

In [8]:
series.resample('30S').bfill()[0:5]

2019-01-01 00:00:00    0
2019-01-01 00:00:30    1
2019-01-01 00:01:00    1
2019-01-01 00:01:30    2
2019-01-01 00:02:00    2
Freq: 30S, dtype: int64

Pass a custom function via apply

In [9]:
def custom_resampler(array_like):
    return np.sum(array_like) + 5

In [10]:
series.resample('3T').apply(custom_resampler)

2019-01-01 00:00:00     8
2019-01-01 00:03:00    17
2019-01-01 00:06:00    18
Freq: 3T, dtype: int64

For a Series with a PeriodIndex, the keyword convention can be used to control whether<br>
to use the start or end of rule.

Resample a year by quarter using ‘start’ convention. Values are assigned to the first<br>
quarter of the period.

In [11]:
s = pd.Series([1, 2], index=pd.period_range('2018-01-01',
                                            freq='A',
                                            periods=2))
s

2018    1
2019    2
Freq: A-DEC, dtype: int64

In [12]:
s.resample('Q', convention='start').asfreq()

2018Q1    1.0
2018Q2    NaN
2018Q3    NaN
2018Q4    NaN
2019Q1    2.0
2019Q2    NaN
2019Q3    NaN
2019Q4    NaN
Freq: Q-DEC, dtype: float64

Resample quarters by month using ‘end’ convention. Values are assigned to the last month of the period.

In [13]:
q = pd.Series([2, 3, 4, 5], index=pd.period_range('2019-01-01',
                                                  freq='Q',
                                                  periods=4))
q

2019Q1    2
2019Q2    3
2019Q3    4
2019Q4    5
Freq: Q-DEC, dtype: int64

In [14]:
q.resample('M', convention='end').asfreq()

2019-03    2.0
2019-04    NaN
2019-05    NaN
2019-06    3.0
2019-07    NaN
2019-08    NaN
2019-09    4.0
2019-10    NaN
2019-11    NaN
2019-12    5.0
Freq: M, dtype: float64

For DataFrame objects, the keyword on can be used to specify the column instead,<br>
of the index for resampling.

In [12]:
d = dict({'price': [8, 9, 7, 11, 12, 16, 15, 17],
          'volume': [40, 50, 30, 80, 40, 80, 30, 40]})
df = pd.DataFrame(d)

In [13]:
df['week_starting'] = pd.date_range('01/01/2019',
                                    periods=8,
                                    freq='W')
df

,price,volume,week_starting
0,8,40,2019-01-06
1,9,50,2019-01-13
2,7,30,2019-01-20
3,11,80,2019-01-27
4,12,40,2019-02-03
5,16,80,2019-02-10
6,15,30,2019-02-17
7,17,40,2019-02-24


In [14]:
df_new = df.resample('M', on='week_starting').mean()

In [15]:
df_new

,price,volume
week_starting,,
2019-01-31,8.75,50.0
2019-02-28,15.00,47.5


In [11]:
type(df_new['price'][0])

numpy.float64

For a DataFrame with MultiIndex, the keyword level can be used to specify on which level<br>
the resampling needs to take place.

In [18]:
days = pd.date_range('1/1/2019', periods=4, freq='D')
d2 = dict({'price': [8, 9, 7, 11, 12, 16, 15, 17],
           'volume': [40, 50, 30, 80, 40, 80, 30, 40]})

In [19]:
df2 = pd.DataFrame(d2,
                   index=pd.MultiIndex.from_product([days,
                                                    ['morning',
                                                     'afternoon']]
                                                    ))
df2

price  volume
2019-01-01 morning        8      40
           afternoon      9      50
2019-01-02 morning        7      30
           afternoon     11      80
2019-01-03 morning       12      40
           afternoon     16      80
2019-01-04 morning       15      30
           afternoon     17      40

In [20]:
df2.resample('D', level=0).sum()

,price,volume
2019-01-01,17,90
2019-01-02,18,110
2019-01-03,28,120
2019-01-04,32,70
